In [1]:

#Primer paso que se realizo fue importar todas las librerias que vamos a utilizar
import pandas as pd
import json
import re
from fastapi import FastAPI
from datetime import datetime

In [3]:
df=pd.read_csv(r"C:\Users\Equipo\Downloads\credits.csv")


In [4]:
df2=pd.read_csv(r"C:\Users\Equipo\Downloads\movies_dataset (1).csv")

C:\Users\Equipo\AppData\Local\Temp\ipykernel_4796\705048831.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv(r"C:\Users\Equipo\Downloads\movies_dataset (1).csv")


In [2]:
#Se crea la funcion para desanidar.
class desanidar:
    @staticmethod
    def convertir_a_str(valor):
        if isinstance(valor, (list, dict)):
            return json.dumps(valor)
        return str(valor)
    @staticmethod
    def extraer_nombres(valor):
        pattern = r"'name': '([^']*)'"
        coincidencias = re.findall(pattern, valor)
        if len(coincidencias) > 0:
            nombre = coincidencias[0]
            return nombre
        else:
            return None

In [7]:
#Segundo paso para desanidar (Son varios)
df2['belongs_to_collection'] = df2['belongs_to_collection'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [8]:
df2['genres'] = df2['genres'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [11]:
df2['production_companies'] = df2['production_companies'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [10]:
df2['production_countries'] = df2['production_countries'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [9]:
df2['spoken_languages'] = df2['spoken_languages'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [13]:
df['cast'] = df['cast'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [14]:
df['crew'] = df['crew'].apply(desanidar.convertir_a_str).apply(desanidar.extraer_nombres)

In [ ]:
df.head(5)

In [ ]:
df2.head(3)

In [19]:
#Se borran las columnas que no sirven a futuro
df2 = df2.dropna(subset=['release_date'])
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df2 = df2.drop(columns=columnas_a_eliminar)

In [23]:
# Convertir las columnas 'budget' y 'revenue' a tipo numérico
df2['budget'] = pd.to_numeric(df2['budget'], errors='coerce')
df2['revenue'] = pd.to_numeric(df2['revenue'], errors='coerce')

# Calcular el retorno de inversión y asignar el valor 0 cuando no haya datos disponibles
df2['return'] = df2['revenue'].div(df2['budget'], fill_value=0)

In [24]:
#Cambio a numerico el tipo de datos de la columna ID
df2['id'] = pd.to_numeric(df2['id'], errors='coerce')

In [ ]:
#Observo cuales son las columnas sin ID
df2[df2['id'].isnull()]

In [26]:
#Dado que solo son 3, decido eliminarlas
df2 = df2.dropna(subset=['id'])

In [27]:
#Cambio el tipo de dato de float a int de la columna, para que coincida en ambos df
df2['id'] = df2['id'].astype(int)

In [29]:
#Realizo un merge entre los dos datasets que tenemos
df_final= pd.merge(df2, df, on='id')

In [30]:
#Se renombra las columnas necesarias
df_final.rename(columns={'crew':'director'}, inplace=True)
df_final.rename(columns={'cast':'actors'}, inplace=True)
df_final.rename(columns={'release_date':'release_year'}, inplace=True)

In [31]:
#Borre la columna belongs_to_collection
df_final.drop('belongs_to_collection', axis=1, inplace=True)

In [34]:
#Guardado de precaución
df_final.to_csv('Csv_Proyecto_Terminado2.csv', index=False)

In [ ]:
# Agregar una nueva columna "año" con el año extraído de "release_year"
for index, row in df.iterrows():
    fecha_completa = row["release_year"]
    año = fecha_completa.split("-")[0]
    df.at[index, "anio"] = int(año)

In [5]:
#Guardado de precaución
df.to_csv('Csv_Proyecto_Terminado.csv', index=False)